<a href="https://colab.research.google.com/github/zyythn/Assignment-MV/blob/main/q4_exponentialdecay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

import torchvision
from torchvision import datasets, models
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
transform = transforms.Compose(
    [transforms.Resize((224,224)),
     transforms.ToTensor(), # convert to 4d-tensor
     transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])]
)

train_dir = '/content/drive/MyDrive/MV/tom and jerry/train'
test_dir = '/content/drive/MyDrive/MV/tom and jerry/test'

train_data = datasets.ImageFolder(root=train_dir,
                                  transform=transform)
test_data = datasets.ImageFolder(root=test_dir,
                                  transform=transform)

In [ ]:
class_names = train_data.classes
class_names

['jerry', 'no tom jerry', 'tom', 'tom jerry']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data,batch_size=4, shuffle=True)

test_dataloader = DataLoader(test_data,batch_size=4, shuffle=False)

In [ ]:
# Define our model
class CNNmodel(nn.Module):
  def __init__(self):
    super(CNNmodel,self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5) #36x36x6
    self.maxpool1 = nn.MaxPool2d(kernel_size=2,stride=2) #18x18x6
    self.conv2 = nn.Conv2d(6,16,5) #14x14x16
    self.maxpool2 = nn.MaxPool2d(kernel_size=2,stride=2) #7x7x16
    self.conv3 = nn.Conv2d(16, 20, 3) #5x5x20
    self.fc1 = nn.Linear(52020,20*5*5)
    self.fc2 = nn.Linear(20*5*5,4)
    self.relu = nn.ReLU()
    self.flatten = nn.Flatten()
    self.batchnorm1 = nn.BatchNorm2d(6)
    self.batchnorm2 = nn.BatchNorm2d(16)
    self.dropout = nn.Dropout(0.4) #drop 40% of neurons during training

  def forward(self,x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.batchnorm1(x)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.batchnorm2(x)
    x = self.relu(x)
    x = self.maxpool2(x)
    x = self.conv3(x)
    x = self.relu(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout(x)
    x = self.relu(x)
    out = self.fc2(x)

    return out

In [ ]:
model = CNNmodel()

In [ ]:
# cross-entropy loss
loss_fn = nn.CrossEntropyLoss()
loss_fn.to('cuda')
optimizer = torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [ ]:
model.to('cuda')

CNNmodel(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(16, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=52020, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=4, bias=True)
  (relu): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (batchnorm1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.4, inplace=False)
)

In [ ]:
# Learning rate scheduler - Exponential Decay
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

#Exponential Learning Rate
lr = []

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
  print("Epoch: {}/{}".format(epoch+1, num_epochs))

  model.train()

  train_loss = 0.0
  train_acc = 0.0

  valid_loss = 0.0
  valid_acc = 0.0

  for inputs, labels in train_dataloader:

    inputs = inputs.to('cuda')
    labels = labels.to('cuda')

    # Clean existing gradients
    optimizer.zero_grad()

    # Forward pass - compute outputs on input data using the model
    outputs = model(inputs)

    # Compute loss
    loss = loss_fn(outputs, labels)

    # Backpropagate the gradients
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Compute the total loss for the batch and add it to train_loss
    train_loss += loss.item() * inputs.size(0)

    # Compute the accuracy
    ret, predictions = torch.max(outputs.data, 1)
    correct_counts = predictions.eq(labels.data.view_as(predictions))

    # Convert correct_counts to float and then compute the mean
    acc = torch.mean(correct_counts.type(torch.FloatTensor))

    # Compute total accuracy in the whole batch and add to train_acc
    train_acc += acc.item() * inputs.size(0)

  # Validation - No gradient tracking needed
  with torch.no_grad():

    model.eval()

    # Validation loop
    for inputs, labels in test_dataloader:
      inputs = inputs.to('cuda')
      labels = labels.to('cuda')

      # Forward pass - compute outputs on input data using the model
      outputs = model(inputs)

      # Compute loss
      loss = loss_fn(outputs, labels)

      # Compute the total loss for the batch and add it to valid_loss
      valid_loss += loss.item() * inputs.size(0)

      # Calculate validation accuracy
      ret, predictions = torch.max(outputs.data, 1)
      correct_counts = predictions.eq(labels.data.view_as(predictions))

      # Convert correct_counts to float and then compute the mean
      acc = torch.mean(correct_counts.type(torch.FloatTensor))

      # Compute total accuracy in the whole batch and add to valid_acc
      valid_acc += acc.item() * inputs.size(0)


  # Find average training loss and training accuracy
  avg_train_loss = train_loss / len(train_dataloader.dataset)
  avg_train_acc = train_acc / len(train_dataloader.dataset)

  # Find average validation loss and training accuracy
  avg_test_loss = valid_loss / len(test_dataloader.dataset)
  avg_test_acc = valid_acc / len(test_dataloader.dataset)

  # Update the learning rate
  scheduler.step()

  lr.append(scheduler.get_last_lr())

  print("Epoch : {:03d}, Training: Accuracy: {:.4f}%, \n\t\tValidation : Accuracy: {:.4f}%".format(epoch, avg_train_acc * 100, avg_test_acc * 100))

Epoch: 1/10
Epoch : 000, Training: Accuracy: 48.4690%, 
		Validation : Accuracy: 34.4073%
Epoch: 2/10
Epoch : 001, Training: Accuracy: 73.0862%, 
		Validation : Accuracy: 35.6404%
Epoch: 3/10
Epoch : 002, Training: Accuracy: 83.9645%, 
		Validation : Accuracy: 42.7208%
Epoch: 4/10
Epoch : 003, Training: Accuracy: 91.8211%, 
		Validation : Accuracy: 38.4248%
Epoch: 5/10
Epoch : 004, Training: Accuracy: 94.0774%, 
		Validation : Accuracy: 35.8393%
Epoch: 6/10
Epoch : 005, Training: Accuracy: 95.2458%, 
		Validation : Accuracy: 36.6348%
Epoch: 7/10
Epoch : 006, Training: Accuracy: 96.5351%, 
		Validation : Accuracy: 39.0215%
Epoch: 8/10
Epoch : 007, Training: Accuracy: 97.0185%, 
		Validation : Accuracy: 38.7033%
Epoch: 9/10
Epoch : 008, Training: Accuracy: 97.5020%, 
		Validation : Accuracy: 38.4646%
Epoch: 10/10
Epoch : 009, Training: Accuracy: 97.7438%, 
		Validation : Accuracy: 37.4702%
